In [62]:
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import joblib
import pandas as pd

In [63]:
f_embeddings = open("./twitter-datasets/twitter-datasets/glove.6B.100d.txt")
embeddings = {}

for line in f_embeddings:
    values = line.split()
    word = values[0]
    vector = np.array(values[1:])
    embeddings[word] = vector

In [64]:
DIM = 100

def load_embeddings(filename):
    f = open(filename)
    pos = []
    original = []
    for line in f:
        values = line[:len(line)-1].split(' ')
        original.append(line)
        values = [val for val in values if val != '<user>' and val in embeddings.keys()]
        for i in range(len(values)):
            if values[i][0] == '#':
                values[i] = values[i][1:]
        values = [val for val in values if val in embeddings.keys()]
        ls = np.zeros((len(values), DIM))
        for i in range(len(values)):
            ls[i] = embeddings[values[i]]
        #print(ls.shape)
        mean = np.mean(ls, axis=0)
        #print(mean.shape)
        pos.append(mean)
    return np.array(pos), original
pos, original_pos = load_embeddings("./twitter-datasets/twitter-datasets/train_pos.txt")
#print(pos)


/home/paul/miniconda3/envs/ml4g/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/paul/miniconda3/envs/ml4g/lib/python3.10/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [65]:
neg, original_neg = load_embeddings("./twitter-datasets/twitter-datasets/train_neg.txt")

In [66]:
vectors = np.concatenate((pos, neg), axis=0)
original = np.concatenate((original_pos, original_neg), axis=0)
labels = np.concatenate((np.ones((pos.shape[0], 1)), np.zeros((neg.shape[0], 1))), axis=0)
labels = labels[~np.isnan(vectors).any(axis=1)]
vectors = vectors[~np.isnan(vectors).any(axis=1)]

In [67]:
np.save("vectors", vectors)
np.save("labels", labels)
np.save("original", original)

In [68]:
vectors = np.load("vectors.npy")
labels = np.load("labels.npy")
original = np.load("original.npy")

In [69]:
indices = np.arange(len(labels))
X_train, X_test, y_train, y_test, _, test_indices = train_test_split(vectors, labels, indices, test_size=0.2, shuffle=True)
#train_data = np.concatenate((X_train, y_train), axis=1)
#np.random.shuffle(train_data)
#X_train = train_data[:, :-1]
#y_train = train_data[:, -1]

In [70]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_classifier.fit(X_train, y_train)

/tmp/ipykernel_251438/1788354763.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [71]:
joblib.dump(rf_classifier, "forest.joblib")

['forest.joblib']

In [72]:
rf_classifier = joblib.load("forest.joblib")

In [73]:
y_pred = rf_classifier.predict(X_test)

In [74]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7491865645492316


In [75]:
y_test = y_test.flatten()
y_pred = y_pred.flatten()

In [88]:
NB = 1000
wrong_mask = y_test[:NB] != y_pred[:NB]
y_actual = y_test[:NB][wrong_mask]
X_actual = original[test_indices[:NB][wrong_mask]]
indices_actual = test_indices[:NB][wrong_mask]
for i in range(200):
    print(indices_actual[i], " actual: ", y_actual[i], " ", X_actual[i])

91899  actual:  1.0   listening to <user> makes the varsity drive so much more entertaining !

36344  actual:  1.0   <user> okay sounds good to me

14635  actual:  1.0   if she wants it she'll get it on her own

23138  actual:  1.0   <user> <user> <user> <user> you ain't k.ow i could shake it like that did ya ? lol

39786  actual:  1.0   <user> where ? )

55527  actual:  1.0   <user> hahahah do sound check tickets include meeting the boys or not ?

36967  actual:  1.0   <user> please wave to us lilrockers who are in the centre the back ... behind the sound people ... please can i meet you ! !

101989  actual:  0.0   ten angry reporters meltingdown & blowing up ( compilation <url> via <user>

58474  actual:  1.0   i'm nt single , n i'm not taken . i'm simply on reserve for the one who deserves my heart.cause they say good things take time .

158595  actual:  0.0   private practice is a major cryfest omggg ( ( (

101055  actual:  0.0   you know who seen me cry the most #mymothermyworld s